In [10]:
### fig 3a

system('gunzip *')

rm(list = ls())  

options(warn=-1)

library(Peptides)

# a function returning per residue average hydrophobicity for a group of sequences of varying length as well as the number of sequences counted at each position
# seqs: a list of sequences
# L_max: take the last L_max residues for very long sequences
# L_min: only consider sequences at least L_min aa long.
# if a sequence is shorter than L_max, fill the beginning with 'x', which will not be used for calculating average hydrophobicity at that position
avg_hydrophobicity_per_position <-function(seqs,L_max,L_min,hydrophobicity_scale){
  H <- rep(0,L_max) # hydrophobicity at each position
  N <- rep(0,L_max) # count of sequence at each position
  i=0
  for(seq in seqs){
    # print progress for every 1000
    i=i+1
    if (i%%1000==0) {
      print(i)
    }
    
    if(nchar(seq) >= L_min){
      # extend the N-terminal with x
      seq = paste(paste(rep('x',L_max),collapse=''),seq,sep='')
      seq = substr(seq,nchar(seq)-L_max+1,nchar(seq))
      
      hi = hydrophobicity(unlist(strsplit(seq,'')),hydrophobicity_scale) 
      H = H + hi 
      N = as.numeric(N) + as.numeric(hi!=0)
    }
  }
  H = H/N
  return(list(H,N))
}

# same as above but look at N-term
avg_hydrophobicity_per_position_nterm <-function(seqs,L_max,L_min,hydrophobicity_scale){
  H <- rep(0,L_max) # hydrophobicity at each position
  N <- rep(0,L_max) # count of sequence at each position
  i=0
  for(seq in seqs){
    # print progress for every 1000
    i=i+1
    if (i%%1000==0) {
      print(i)
    }
    
    if(nchar(seq) >= L_min){
      seq = paste(seq,paste(rep('x',L_max),collapse=''),sep='') # changed
      seq = substr(seq,1,L_max) # changed
      hi = hydrophobicity(unlist(strsplit(seq,'')),hydrophobicity_scale) 
      H = H + hi 
      N = as.numeric(N) + as.numeric(hi!=0)
    }
  }
  H = H/N
  return(list(H,N))
}

L_max=100
L_min=10
hydrophobicity_scale = 'Miyazawa' 
#hydrophobicity_scale = 'Ponnuswamy'
saved_data = paste('genome-scale-hydrophobicity-',hydrophobicity_scale,'-100aa.RData',sep='')
  
if(! file.exists(saved_data)){
    
  # for canonical proteome (cds), only consider those >200 aa
  # TBD: some gene has mulitple isoforms. currently considers all
  # more human.CDS.fa-proteome.fa | grep -v ">" | sed 's/^M//g'  | sed 's/*$//g' | sort | uniq | awk 'length($0)>200' > cds-200aa.txt
  x=read.table('cds-200aa.txt')
  cds = avg_hydrophobicity_per_position(x$V1,L_max,L_min,hydrophobicity_scale)
  
  # for 5' UTR: take the longest ORF, remove M and * at beginning and end. then remove those found in the canoical proteome
  # more human.mRNA.fa-utr5-proteome-longest.fa | grep -v ">" | sed 's/^M//g'  | sed 's/*$//g' | sort | uniq > utr5.longest.txt
  # python grep_between_files.py utr5.longest.txt human.CDS.fa-proteome.fa
  # mv utr5.longest.txt-not-matched.txt utr5.longest.txt
  x=read.table('utr5.longest.txt')
  utr5 = avg_hydrophobicity_per_position(x$V1,L_max,L_min,hydrophobicity_scale)
  
  ## UTR3
  # more human.mRNA.fa-utr3-proteome-frame0.fa | grep -v ">" | sed 's/^M//g'  | sed 's/*$//g' | awk 'length($0)>=10' | sort | uniq > utr3.10aa.txt
  x=read.table('utr3.10aa.txt')
  utr3 = avg_hydrophobicity_per_position(x$V1,L_max,L_min,hydrophobicity_scale)
  
  # longest.lncrna.peptide.txt
  # more human.lncRNA.fa-lncrna-proteome-longest.fa | grep -v ">" | sed 's/^M//g'  | sed 's/*$//g' | sort | uniq > lncrna-logest.txt
  # python grep_between_files.py lncrna-logest.txt human.CDS.fa-proteome.fa
  # mv lncrna-logest.txtnot-matched.txt lncrna-longest.txt
  x=read.table('lncrna-longest.txt')
  lncrna = avg_hydrophobicity_per_position(x$V1,L_max,L_min,hydrophobicity_scale)
  
  # intron peptide: all 3 frames considered, >=30aa, uniq
  # more human.intron.fa-intron-proteome.fa | grep -v ">" | sed 's/^M//g'  | sed 's/*$//g' | awk 'length($0)>=30' | sort | uniq > intron.30aa.txt
  # python grep_between_files.py intron.30aa.txt human.CDS.fa-proteome.fa
  # mv intron.30aa.txtnot-matched.txt intron.30aa.txt
  x=read.table('intron.30aa.txt')
  intron = avg_hydrophobicity_per_position(x$V1,L_max,L_min,hydrophobicity_scale)
  save.image(file=saved_data)
} else {
  load(file=saved_data)
}

# plotting: need to adjust ylim if using a different scale

if (hydrophobicity_scale == 'Miyazawa'){
  ylim=c(5.25,5.7)
} else if (hydrophobicity_scale == 'Ponnuswamy'){
  ylim=c(0.35,0.65)
}

dev.off()

pdf(paste('fig3a-',hydrophobicity_scale,'.pdf',sep=''),width=3,height=4)

L=100
skip_last=2
x=-L:-1
y=cds[[1]][(L_max-L+1):L_max]
lo <- loess(y[1:(L-skip_last)]~x[1:(L-skip_last)])
plot(x,y,ylim=ylim,pch=16,cex=0.2,col='red',bty='l',ylab='\nAverage hydrophobicity',xlab='Position relative to C-termini\n')
lines(x, predict(lo,x), col='red', lwd=2)

y=utr5[[1]][(L_max-L+1):L_max]
lo <- loess(y[1:(L-skip_last)]~x[1:(L-skip_last)])
points(x,y,col='blue',pch=16,cex=0.2)
lines(x, predict(lo,x), col='blue', lwd=2)

y=utr3[[1]][(L_max-L+1):L_max]
lo <- loess(y[1:(L-skip_last)]~x[1:(L-skip_last)])
points(x,y,col='orange',pch=16,cex=0.2)
lines(x, predict(lo,x), col='orange', lwd=2)

y=lncrna[[1]][(L_max-L+1):L_max]
lo <- loess(y[1:(L-skip_last)]~x[1:(L-skip_last)])
points(x,y,col='green',pch=16,cex=0.2)
lines(x, predict(lo,x), col='green', lwd=2)

y=intron[[1]][(L_max-L+1):L_max]
lo <- loess(y[1:(L-skip_last)]~x[1:(L-skip_last)])
points(x,y,col='magenta',pch=16,cex=0.2)
lines(x, predict(lo,x), col='magenta', lwd=2)

abline(a=hydrophobicity('ACDEFGHIKLMNPQRSTVWY',hydrophobicity_scale)  ,b=0,lty=2,lwd=2,col='gray')
legend('topleft',"Random",text.col='gray',bty='n')
dev.off()

#legend(0,0,legend=c("CDS","5'UTR/uORF","3'UTR","LncRNA","Intron"),text.col=c('red','blue','orange','green','magenta'),bty='n')


system('gzip *.txt')

[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000
[1] 9000
[1] 10000
[1] 11000
[1] 12000
[1] 13000
[1] 14000
[1] 15000
[1] 16000
[1] 17000
[1] 18000
[1] 19000
[1] 20000
[1] 21000
[1] 22000
[1] 23000
[1] 24000
[1] 25000
[1] 26000
[1] 27000
[1] 28000
[1] 29000
[1] 30000
[1] 31000
[1] 32000
[1] 33000
[1] 34000
[1] 35000
[1] 36000
[1] 37000
[1] 38000
[1] 39000
[1] 40000
[1] 41000
[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000
[1] 9000
[1] 10000
[1] 11000
[1] 12000
[1] 13000
[1] 14000
[1] 15000
[1] 16000
[1] 17000
[1] 18000
[1] 19000
[1] 20000
[1] 21000
[1] 22000
[1] 23000
[1] 24000
[1] 25000
[1] 26000
[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000
[1] 9000
[1] 10000
[1] 11000
[1] 12000
[1] 13000
[1] 14000
[1] 15000
[1] 16000
[1] 17000
[1] 18000
[1] 19000
[1] 20000
[1] 21000
[1] 22000
[1] 23000
[1] 24000
[1] 25000
[1] 26000
[1] 27000
[1] 28000
[1] 29000
[1] 30000
[1] 31000
[1] 32000
[1] 1000
[1] 2000
[1] 3000
[1] 4000
[

null device 
          1

null device 
          1